# Read Me

- 기존 데이터 셋으로 학습 시켰을 때 성능이 좋지 않아, 이를 개선하기 위해 많은 원인을 분석해 보았음.
- 이 과정은 많은 원인 중에 하나로 모델을 개선시키기 위한 노력임.

- yolov 모델의 학습에 있어서 **bounding box의 크기는 매우 중요함.**
    - bounding box가 클 때는 손실이 발생하더라도 이미지를 잘 파악할 수 있지만,
    - 매우 작은 bouding box는 손실이 발생했을 때 이미지를 잘 따라갈 수 없다는 한계가 있음.
    - 이에 bbox 관련 형성된 txt 파일을 불러와서 분포를 확인하고, 일정 값 보다 작은 bbox들은 삭제해줄 것임

In [15]:
import cv2
import numpy as np
import glob
import pandas as pd

import random
import shutil
import os
import zipfile
from tqdm import tqdm

import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# 1. bbox의 크기 분포 확인해주기

In [3]:
import os

def get_bboxes_sizes_from_files(txt_files_dir):
    sizes = []
    
    # txt 파일들을 순회하면서 bbox 크기 저장
    for filename in os.listdir(txt_files_dir):
        if filename.endswith(".txt"):
            txt_file_path = os.path.join(txt_files_dir, filename)
            
            # txt 파일 열기
            with open(txt_file_path, "r") as f:
                lines = f.readlines()
                
                # 각 라인에서 bbox 크기 추출
                for line in lines:
                    bbox_info = line.strip().split(" ")
                    if len(bbox_info) == 5:
                        _, x, y, w, h = bbox_info
                        bbox_width = float(w)
                        bbox_height = float(h)
                        
                        sizes.append((bbox_width, bbox_height))
    
    return sizes

# txt 파일들이 위치한 디렉토리 경로
txt_files_dir = "C:/Users/acorn/Desktop/damage_train_images_97500/labels"

# bbox 크기 분포 확인
bbox_sizes = get_bboxes_sizes_from_files(txt_files_dir)

# bbox 크기 분포 출력
print("bbox 크기 분포:")
for size in bbox_sizes:
    print("Width:", size[0], "Height:", size[1])

bbox 크기 분포:
Width: 0.1925 Height: 0.48
Width: 0.0725 Height: 0.17333333333333334
Width: 0.14682539682539683 Height: 0.12169312169312169
Width: 0.34125 Height: 0.51
Width: 0.29 Height: 0.2683333333333333
Width: 0.40125 Height: 0.26
Width: 0.0984375 Height: 0.14479166666666668
Width: 0.29625 Height: 0.45166666666666666
Width: 0.0725 Height: 0.115
Width: 0.72625 Height: 0.6733333333333333
Width: 0.2925 Height: 0.775
Width: 0.08875 Height: 0.08166666666666667
Width: 0.09125 Height: 0.15166666666666667
Width: 0.78625 Height: 0.29
Width: 0.3 Height: 0.36833333333333335
Width: 0.03125 Height: 0.07666666666666666
Width: 0.06375 Height: 0.13
Width: 0.06375 Height: 0.06
Width: 0.03875 Height: 0.035
Width: 0.05625 Height: 0.03666666666666667
Width: 0.09375 Height: 0.04833333333333333
Width: 0.24125 Height: 0.18833333333333332
Width: 0.735 Height: 0.8083333333333333
Width: 0.73 Height: 0.205
Width: 0.3575 Height: 0.125
Width: 0.0175 Height: 0.03
Width: 0.19875 Height: 0.2633333333333333
Width: 0.1

Width: 0.04 Height: 0.06
Width: 0.01625 Height: 0.03166666666666667
Width: 0.095 Height: 0.16833333333333333
Width: 0.19125 Height: 0.26166666666666666
Width: 0.12125 Height: 0.205
Width: 0.0525 Height: 0.021666666666666667
Width: 0.24375 Height: 0.09333333333333334
Width: 0.0375 Height: 0.03333333333333333
Width: 0.01 Height: 0.015
Width: 0.19875 Height: 0.035
Width: 0.10125 Height: 0.06333333333333334
Width: 0.015 Height: 0.015
Width: 0.0125 Height: 0.016666666666666666
Width: 0.1075 Height: 0.035
Width: 0.015 Height: 0.11333333333333333
Width: 0.01 Height: 0.013333333333333334
Width: 0.0125 Height: 0.013333333333333334
Width: 0.0875 Height: 0.01
Width: 0.075 Height: 0.011666666666666667
Width: 0.04625 Height: 0.09333333333333334
Width: 0.02375 Height: 0.041666666666666664
Width: 0.0225 Height: 0.045
Width: 0.00875 Height: 0.016666666666666666
Width: 0.01125 Height: 0.018333333333333333
Width: 0.17125 Height: 0.23
Width: 0.08 Height: 0.11166666666666666
Width: 0.4925 Height: 0.368333

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
bbox = pd.DataFrame({"bbox":bbox_sizes})

In [16]:
bbox_multiple = []

for i in range(len(bbox_sizes)):
    bbox_multiple.append(bbox_sizes[i][0] * bbox_sizes[i][1])

In [8]:
len(bbox)

126482

In [17]:
len(bbox_multiple)

126482

In [18]:
bbox['multiple'] = bbox_multiple

In [19]:
bbox

,bbox,multiple
0,"(0.1925, 0.48)",0.092400
1,"(0.0725, 0.17333333333333334)",0.012567
2,"(0.14682539682539683, 0.12169312169312169)",0.017868
3,"(0.34125, 0.51)",0.174038
4,"(0.29, 0.2683333333333333)",0.077817
...,...,...
126477,"(0.03, 0.04666666666666667)",0.001400
126478,"(0.3225, 0.26)",0.083850
126479,"(0.015932521087160263, 0.02666666666666667)",0.000425
126480,"(0.05529522024367385, 0.055)",0.003041


In [22]:
bbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126482 entries, 0 to 126481
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   bbox      126482 non-null  object 
 1   multiple  126482 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.9+ MB


In [35]:
bbox['multiple'].describe()

count    126482.000000
mean          0.073464
std           0.129468
min           0.000006
25%           0.004010
50%           0.019350
75%           0.079231
max           1.000000
Name: multiple, dtype: float64

In [34]:
bbox['multiple'].mean()

0.07346354082858343

# 2.bbox 그려서 확인

In [ ]:
txt_dir = "C:/Users/acorn/Desktop/damage_train_images_97500/labels"  
img_dir = "C:/Users/acorn/Desktop/damage_train_images_97500/damage"  

txt_files = [file_name for file_name in os.listdir(txt_dir) if file_name.endswith(".txt")]

for txt_file in txt_files:
    # txt 파일과 동일한 이름명을 가진 jpg파일 불러오기
    img_file = os.path.splitext(txt_file)[0] + ".jpg"  
    
    # 이미지 불러오기
    image_path = os.path.join(img_dir, img_file)
    image = Image.open(image_path)

    # txt파일에서 bbox 정보 불러오기
    with open(os.path.join(txt_dir, txt_file), "r") as f:
        lines = f.readlines()

    box_coordinates = []
    for line in lines:
        x, y, width, height = map(float, line.split()[1:])  # 각 줄에서 x, y, w, h 불러오기
        xtl = (x - width / 2) * image.width
        ytl = (y - height / 2) * image.height
        xbr = (x + width / 2) * image.width
        ybr = (y + height / 2) * image.height
        box_coordinates.append([xtl, ytl, xbr, ybr])

    # 이미지 위에 bbox 그리기
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    for box in box_coordinates:
        xtl, ytl, xbr, ybr = box
        x = xtl
        y = ytl
        width = xbr - xtl
        height = ybr - ytl
        rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor="r", facecolor="none")
        ax.add_patch(rect)

C:\Users\acorn\AppData\Local\Temp\ipykernel_21404\1746065236.py:36: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(1)


# 3. 크기가 너무 작은 bbox들은 yolov 모델 학습 성능을 줄일 수 있기에 삭제

## 3-1. Txt 파일 하나씩 열람하면서, bbox가 작은 것들을 삭제

- width * height 의 값이 1분위수(25%) 값인 0.004보다 작은 것들을 삭제
- 원본을 보존

In [ ]:
origin_img_dir = "C:/Users/acorn/Desktop/damage_train_images_97500/damage"
origin_txt_dir = "C:/Users/acorn/Desktop/damage_train_images_97500/labels"

dst_img_dir = "C:/Users/acorn/Desktop/damage_valid_labels_30000/images"  
dst_txt_dir = "C:/Users/acorn/Desktop/damage_valid_labels_30000/labels"  

# 원본 폴더의 모든 파일을 가져오기
image_file_list = os.listdir(origin_img_dir)
txt_file_list = os.listdir(origin_txt_dir)

# 이미지 파일부터 복사
for file_name in image_file_list:
    # 파일의 전체 경로 생성
    sep = os.path.join(origin_img_dir, file_name)
    dst = os.path.join(dst_img_dir, file_name)
    
    # 파일 복사
    shutil.copy(sep, dst)
    
    
for file_name in txt_file_list:
    # 파일의 전체 경로 생성
    sep = os.path.join(origin_txt_dir, file_name)
    dst = os.path.join(dst_txt_dir, file_name)
    
    # 파일 복사
    shutil.copy(sep, dst)

print("파일 복사가 완료되었습니다.")

In [2]:
directory = "C:/Users/acorn/Desktop/damage_valid_labels_30000/labels"
threshold = 0.004

# 디렉토리 내의 모든 txt 파일 가져오기
txt_files = [file for file in os.listdir(directory) if file.endswith(".txt")]

for txt_file in tqdm(txt_files):
    file_path = os.path.join(directory, txt_file)
    
    with open(file_path, "r") as file:
        lines = file.readlines()

    # w * h 값이 threshold보다 작은 줄 삭제하기
    lines = [line for line in lines if float(line.split()[3]) * float(line.split()[4]) >= threshold]

    with open(file_path, "w") as file:
        file.writelines(lines)

    # 줄이 줄어든 경우 마지막 공백 제거
    with open(file_path, "r") as file:
        lines = file.readlines()

    lines = [line.strip() for line in lines]

    with open(file_path, "w") as file:
        file.write("\n".join(lines))

100%|█████████████████████████████████████████████████████████████████████████████| 6000/6000 [00:26<00:00, 229.15it/s]


In [3]:
txt = glob.glob("C:/Users/acorn/Desktop/damage_valid_labels_30000/labels/*.txt")

In [4]:
len(txt)

6000

## 3-2. 줄이 삭제되면서 빈 txt파일이 생겼을 수 있음. 이를 삭제 해주기

- 6000개 -> 5828개로 삭제되었음

In [5]:
directory = "C:/Users/acorn/Desktop/damage_valid_labels_30000/labels"

# 디렉토리 내의 모든 txt 파일 가져오기
txt_files = [file for file in os.listdir(directory) if file.endswith(".txt")]

for txt_file in txt_files:
    file_path = os.path.join(directory, txt_file)
    
    # 파일이 비어있는지 확인
    if os.path.getsize(file_path) == 0:
        os.remove(file_path)

In [6]:
txt = glob.glob("C:/Users/acorn/Desktop/damage_valid_labels_30000/labels/*.txt")

In [7]:
len(txt)

5828

## 3-3. txt파일명과 일치하는 jpg 파일만 남기기

In [8]:
txt_files = glob.glob("C:/Users/acorn/Desktop/damage_valid_labels_30000/labels/*.txt")

total_txt_name_list = []

for i in range(len(txt_files)):
    total_txt_name_list.append(txt_files[i].split("\\")[-1].split(".")[0] + '.jpg') 

In [9]:
delete_path = 'C:/Users/acorn/Desktop/damage_valid_images_30000/images'  

train_file_list = os.listdir(delete_path)

for file in tqdm(train_file_list):
    if file not in total_txt_name_list:
        file_path = os.path.join(delete_path, file)
        os.remove(file_path)
        print(f'파일 삭제: {file_path}')

 55%|█████████████████████████████████████████                                  | 3280/6000 [00:00<00:00, 32657.23it/s]

파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0000795_sc-1038002.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0001234_as-2546236.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0002073_sc-134253.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0003136_sc-158584.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0003205_sc-196663.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0004725_sc-1035563.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0004870_as-0098014.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0005564_as-0052775.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0005573_as-0019881.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0006040_sc-1010294.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0006887_sc-1025183.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/i

100%|███████████████████████████████████████████████████████████████████████████| 6000/6000 [00:00<00:00, 21066.60it/s]

파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0064295_as-0061294.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0064581_as-0064432.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0065123_sc-1040006.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0065552_as-2981987.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0065802_as-0073521.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0066656_as-0004293.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0067422_sc-188554.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0067450_sc-1004741.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0067533_sc-1039807.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0067582_sc-127005.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/images\0067676_sc-121759.jpg
파일 삭제: C:/Users/acorn/Desktop/damage_valid_images_30000/i

## 3-4. set()을 이용하여 img파일명과 jpg파일명의 개수가 동일한지 확인해주기  

In [11]:
valid_image_list = glob.glob("C:/Users/acorn/Desktop/damage_valid_images_30000/images/*.jpg")
valid_total_txt = glob.glob("C:/Users/acorn/Desktop/damage_valid_labels_30000/labels/*.txt")

print(len(valid_image_list))
print(len(valid_total_txt))

5828
5828


In [12]:
image_file_list=[]

for i in range(len(valid_image_list)):
    image_file_list.append(valid_image_list[i].split("\\")[-1].split(".")[0])
    
    
total_txt_list = []

for i in range(len(valid_total_txt)):
    total_txt_list.append(valid_total_txt[i].split("\\")[-1].split(".")[0])
    
image_list_set = set(image_file_list)
total_txt_set = set(total_txt_list)

len(image_list_set & total_txt_set)   # 파일 명 일치

5828

## 3-5. 알집으로 압축해주기

In [18]:
file_pattern = 'C:/Users/acorn/Desktop/damage_valid_images_30000/images/*.jpg'
output_path = 'C:/Users/acorn/Desktop/damage_valid_images_30000/images/damage_valid_images_35000_v2.zip'

# 파일 경로 리스트 가져오기
file_paths = glob.glob(file_pattern)

with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # tqdm을 사용하여 진행 상황 표시
    for file_path in tqdm(file_paths, desc='Compressing files', unit='files'):
        zipf.write(file_path, arcname=file_path.split('/')[-1])

Compressing files: 100%|███████████████████████████████████████████████████████| 5828/5828 [00:17<00:00, 330.48files/s]


In [20]:
file_pattern = 'C:/Users/acorn/Desktop/damage_valid_labels_30000/labels/*.txt'
output_path = 'C:/Users/acorn/Desktop/damage_valid_labels_30000/labels/damage_valid_labels_35000_v2.zip'

# 파일 경로 리스트 가져오기
file_paths = glob.glob(file_pattern)

with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # tqdm을 사용하여 진행 상황 표시
    for file_path in tqdm(file_paths, desc='Compressing files', unit='files'):
        zipf.write(file_path, arcname=file_path.split('/')[-1])

Compressing files: 100%|███████████████████████████████████████████████████████| 5828/5828 [00:19<00:00, 304.06files/s]
